<a href="https://colab.research.google.com/github/luc-leonard/taming-transformers/blob/master/scripts/clip_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTEBOOK PREPARATION


In [2]:
!git clone https://github.com/luc-leonard/taming-transformers
!git clone https://github.com/openai/CLIP

Cloning into 'taming-transformers'...
remote: Enumerating objects: 768, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 768 (delta 55), reused 63 (delta 27), pack-reused 658
Receiving objects: 100% (768/768), 119.50 MiB | 25.19 MiB/s, done.
Resolving deltas: 100% (190/190), done.
Cloning into 'CLIP'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.


In [3]:
!pip install kornia omegaconf pytorch-lightning ftfy

     |████████████████████████████████| 286kB 12.5MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 808kB 25.4MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 112kB 37.6MB/s 
     |████████████████████████████████| 645kB 37.4MB/s 
     |████████████████████████████████| 10.6MB 35.2MB/s 
     |████████████████████████████████| 112kB 40.2MB/s 
     |████████████████████████████████| 829kB 35.0MB/s 
     |████████████████████████████████| 276kB 37.1MB/s 
     |████████████████████████████████| 1.3MB 34.3MB/s 
     |████████████████████████████████| 296kB 37.8MB/s 
     |████████████████████████████████| 143kB 31.6MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=7a8c4167b44192d243da95bd2e88a3f1f293de915eb7c4621fb87d515bb9b64e
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
  Created wheel for a

## DOWNLOADING VQGAN MODEL

In [ ]:
!curl -L 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' > vqgan_imagenet_f16_16384.yaml
!curl -L 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' > vqgan_imagenet_f16_16384.ckpt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   692  100   692    0     0   6178      0 --:--:-- --:--:-- --:--:--  6178
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 19  934M   19  183M    0     0  15.9M      0  0:00:58  0:00:11  0:00:47 14.9M

# HAVE WE BEEN LUCKY ? :)

/!\ DO NOT FORGET TO GO TO 'runtime' => 'change runtime type' and to choose a GPU instance !

In [ ]:
!nvidia-smi

In [ ]:
import sys
sys.path.append('taming-transformers')
sys.path.append('CLIP')

from scripts.clip_generator.trainer import Trainer
from scripts.clip_generator.dreamer import load_vqgan_model
import clip
from PIL import Image

device = 'cuda'

vqgan_model = load_vqgan_model('./vqgan_imagenet_f16_16384.yaml', './vqgan_imagenet_f16_16384.ckpt')
clip_model = clip.load('ViT-B/32', jit=False)[0].eval().requires_grad_(False).to(device)

# RUNNING

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.

prompt = 'A cubic leopard'  #@param {type: "string"}
epochs = 240  #@param {type: "slider", min: 100, max: 1000}
#@markdown ---

from IPython import display

trainer = Trainer([prompt], vqgan_model, clip_model, save_every=10, steps=epochs, device=device)
for _ in trainer.epoch():
  display.clear_output()
  display.display(Image.open('./out/progress_latest.png'))